# 03 – Pitch-Class Histograms (Somali vs. Western)

This notebook estimates **fundamental frequency (F0)** using PYIN, converts F0 to **MIDI**,
then computes **pitch-class histograms** (12 bins modulo octave). It compares two songs via
histogram plots and simple similarity metrics.

**Steps**
1. Load two audio clips (`../data/song_a.wav`, `../data/song_b.wav`)
2. Estimate F0 with `librosa.pyin`
3. Convert Hz → MIDI, take modulo 12 for pitch-class
4. Make 12-bin histograms (+ optional smoothing)
5. Compare distributions (cosine similarity, JS divergence)

> Tip: Keep clips short (15–30s) for fast runs.


In [1]:
# !pip install librosa numpy matplotlib --quiet
import os, numpy as np, librosa, librosa.display, matplotlib.pyplot as plt
from numpy.linalg import norm

# Configure files
file_a = "../data/song_a.wav"
file_b = "../data/song_b.wav"
assert os.path.exists(file_a) and os.path.exists(file_b), "Add song_a.wav and song_b.wav to ../data/"

SR = 22050
y_a, _ = librosa.load(file_a, sr=SR, mono=True)
y_b, _ = librosa.load(file_b, sr=SR, mono=True)
print("Loaded A len=%.2fs  B len=%.2fs" % (len(y_a)/SR, len(y_b)/SR))

AssertionError: Add song_a.wav and song_b.wav to ../data/

In [ ]:
# F0 estimation with PYIN
f0_a, _, _ = librosa.pyin(y_a, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=SR)
f0_b, _, _ = librosa.pyin(y_b, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=SR)

def hz_to_midi(hz):
    hz = np.asarray(hz, dtype=float)
    midi = 69 + 12*np.log2(hz/440.0 + 1e-12)  # add epsilon for safety
    return midi

# Convert to MIDI, drop NaNs (unvoiced)
midi_a = hz_to_midi(f0_a)
midi_b = hz_to_midi(f0_b)
midi_a = midi_a[~np.isnan(midi_a)]
midi_b = midi_b[~np.isnan(midi_b)]
print("Voiced frames: A =", midi_a.size, " B =", midi_b.size)

In [ ]:
# Pitch-class (mod 12), histogram, and smoothing
pc_a = np.mod(midi_a, 12.0)
pc_b = np.mod(midi_b, 12.0)

names = np.array(['C','C#','D','D#','E','F','F#','G','G#','A','A#','B'])
bins = np.arange(13)  # 0..12

hist_a, _ = np.histogram(pc_a, bins=bins)
hist_b, _ = np.histogram(pc_b, bins=bins)

# normalize to probability
p_a = hist_a / (hist_a.sum() + 1e-8)
p_b = hist_b / (hist_b.sum() + 1e-8)

# Optional circular smoothing (window size=3 bins)
def circular_smooth(p, k=1):
    n = len(p)
    q = np.zeros_like(p, dtype=float)
    for i in range(n):
        for d in range(-k, k+1):
            q[i] += p[(i+d) % n]
    q /= q.sum() + 1e-8
    return q

p_a_sm = circular_smooth(p_a, k=1)
p_b_sm = circular_smooth(p_b, k=1)

# Plot histograms
plt.figure(figsize=(10,4))
x = np.arange(12)
plt.bar(x-0.15, p_a_sm, width=0.3, label="A (smoothed)")
plt.bar(x+0.15, p_b_sm, width=0.3, label="B (smoothed)")
plt.xticks(x, names)
plt.ylabel("Probability")
plt.title("Pitch-Class Histogram (12-TET bins)")
plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
# Similarity metrics
def cosine_sim(p, q):
    return float(np.dot(p, q) / (norm(p)*norm(q) + 1e-12))

def js_divergence(p, q):
    # Jensen–Shannon divergence (symmetric, bounded)
    p = p / (p.sum() + 1e-12)
    q = q / (q.sum() + 1e-12)
    m = 0.5*(p+q)
    def kl(a, b):
        a = np.clip(a, 1e-12, 1.0)
        b = np.clip(b, 1e-12, 1.0)
        return float(np.sum(a * np.log(a/b)))
    return 0.5*kl(p, m) + 0.5*kl(q, m)

print("Cosine similarity (smoothed PCs):", round(cosine_sim(p_a_sm, p_b_sm), 4))
print("JS divergence  (smoothed PCs):   ", round(js_divergence(p_a_sm, p_b_sm), 4))

In [4]:
# Example inside Jupyter
from transcribe import run   # if you saved it in a file, otherwise just use the function in your cell

audio_file = "your_song.wav"   # replace with your oud music file
out_csv = "out/transcription.csv"
out_midi = "out/transcription.mid"

run(audio_file, out_csv, out_midi, sr=22050)


/Users/khalidibrahim/Desktop/somali-music-feature-explorer/notebooks/transcribe.py:185: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=sr, mono=True)
/Users/khalidibrahim/Desktop/somali-music-feature-explorer/.venv/lib/python3.13/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'your_song.wav'

In [5]:
# --- Imports ---
import os, numpy as np, librosa
from mido import Message, MidiFile, MidiTrack, MetaMessage, bpm2tempo

# ---------- Helpers ----------
NOTE_NAMES = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
CHROMATIC_SOLFEGE = {0:'do',1:'di',2:'re',3:'ri',4:'mi',5:'fa',6:'fi',7:'so',8:'si',9:'la',10:'li',11:'ti'}
KR_MAJOR = np.array([6.35,2.23,3.48,2.33,4.38,4.09,2.52,5.19,2.39,3.66,2.29,2.88])
KR_MINOR = np.array([6.33,2.68,3.52,5.38,2.60,3.53,2.54,4.75,3.98,2.69,3.34,3.17])

def hz_to_midi(hz):
    hz = np.asarray(hz, dtype=float)
    return 69 + 12 * np.log2(np.maximum(hz, 1e-12) / 440.0)

def midi_to_name(m):
    m = int(round(m)); return f"{NOTE_NAMES[m % 12]}{m//12 - 1}"

def rotate(a, n): return np.roll(a, -n)

def key_from_hist(pc_hist):
    best_r, best = -1e9, ("C","major",0)
    for tonic in range(12):
        rM = np.corrcoef(pc_hist, rotate(KR_MAJOR, tonic))[0,1]
        rN = np.corrcoef(pc_hist, rotate(KR_MINOR, tonic))[0,1]
        if rM > best_r: best_r, best = rM, (NOTE_NAMES[tonic], "major", tonic)
        if rN > best_r: best_r, best = rN, (NOTE_NAMES[tonic], "minor", tonic)
    return best  # (tonic_name, mode, tonic_pc)

def solfege_for_pc(pc, tonic_pc):
    return CHROMATIC_SOLFEGE[int((pc - tonic_pc) % 12)]

# ---------- Core ----------
def track_to_notes(y, sr=22050, fmin='C2', fmax='C7',
                   voicing_prob_thresh=0.5, median_win=5,
                   min_note_ms=80):
    """PYIN → MIDI → median smooth → semitone quantize → note segmentation."""
    f0, vflag, vprob = librosa.pyin(
        y, sr=sr,
        fmin=librosa.note_to_hz(fmin),
        fmax=librosa.note_to_hz(fmax)
    )
    times = librosa.times_like(f0, sr=sr)
    voiced = (vflag == True) & (np.nan_to_num(vprob) >= voicing_prob_thresh)
    f0_clean = np.where(voiced, f0, np.nan)

    midi = hz_to_midi(f0_clean)
    if np.any(~np.isnan(midi)):
        idx = np.flatnonzero(~np.isnan(midi))
        midi_filled = np.where(np.isnan(midi),
                               np.interp(np.flatnonzero(np.isnan(midi)), idx, midi[idx]),
                               midi)
    else:
        midi_filled = np.full_like(midi, 69.0)

    if median_win > 1:
        from scipy.ndimage import median_filter
        midi_smooth = median_filter(midi_filled, size=median_win)
    else:
        midi_smooth = midi_filled
    midi_smooth = np.where(voiced, midi_smooth, np.nan)
    midi_round = np.rint(midi_smooth)

    notes = []
    in_note, start_idx, cur = False, 0, None
    for i in range(len(midi_round)+1):
        same = (i < len(midi_round) and not np.isnan(midi_round[i]) and (cur is None or midi_round[i] == cur))
        if not in_note:
            if i < len(midi_round) and not np.isnan(midi_round[i]):
                in_note, start_idx, cur = True, i, midi_round[i]
        else:
            if not same:
                onset = times[start_idx]
                offset = times[i-1] if i-1 < len(times) else times[-1]
                if (offset - onset)*1000 >= min_note_ms:
                    midi_note = int(cur); pc = midi_note % 12
                    notes.append({"onset": float(onset), "offset": float(offset), "midi": midi_note, "pc": pc})
                in_note, cur = False, None
    return notes, times, midi_smooth

def pitchclass_hist(notes):
    h = np.zeros(12, float); total = 0.0
    for n in notes:
        dur = max(n["offset"]-n["onset"], 1e-3)
        h[n["pc"]] += dur; total += dur
    return h/total if total>0 else h

def detect_key_from_notes(notes):
    if not notes: return ("C","major",0)
    return key_from_hist(pitchclass_hist(notes))

def write_midi(notes, out_path, tempo_bpm=90):
    mid = MidiFile(); tr = MidiTrack(); mid.tracks.append(tr)
    tr.append(MetaMessage('set_tempo', tempo=bpm2tempo(tempo_bpm)))
    tpb = mid.ticks_per_beat
    us_per_beat = 60_000_000 / tempo_bpm
    ticks_per_sec = tpb / (us_per_beat / 1e6)

    def sec2ticks(s): return int(round(s * ticks_per_sec))
    t_prev = 0.0
    for n in notes:
        tr.append(Message('note_on', note=n["midi"], velocity=90, time=sec2ticks(n["onset"]-t_prev)))
        tr.append(Message('note_off', note=n["midi"], velocity=64, time=sec2ticks(n["offset"]-n["onset"])))
        t_prev = n["offset"]
    mid.save(out_path)

def run(audio_path, out_csv="out/transcription.csv", out_midi="out/transcription.mid", sr=22050):
    os.makedirs(os.path.dirname(out_csv) or ".", exist_ok=True)
    os.makedirs(os.path.dirname(out_midi) or ".", exist_ok=True)

    y, sr = librosa.load(audio_path, sr=sr, mono=True)
    notes, times, midi_curve = track_to_notes(y, sr=sr)

    tonic_name, mode, tonic_pc = detect_key_from_notes(notes)

    # CSV
    lines = ["onset_s,offset_s,midi,note,solfege"]
    for n in notes:
        lines.append(f"{n['onset']:.3f},{n['offset']:.3f},{n['midi']},{midi_to_name(n['midi'])},{solfege_for_pc(n['pc'], tonic_pc)}")
    with open(out_csv, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    # MIDI
    write_midi(notes, out_midi, tempo_bpm=90)

    print(f"Detected key: {tonic_name} {mode}")
    print(f"Wrote CSV → {out_csv}")
    print(f"Wrote MIDI → {out_midi}")
    return notes, (tonic_name, mode, tonic_pc)


### Notes
- If A and B use **different scales**, their pitch-class histograms will show different peaks.
- Somali pentatonic patterns may emphasize ~5 pitch classes; Western heptatonic (major/minor) tends to spread across 7.
- Microtonal tendencies won’t be captured perfectly by 12-TET bins; later, consider **cents histograms** (e.g., 24 or 48 bins per octave).
